# Assignment 4 - Naive Machine Translation and LSH

In this notebook, you'll implement a simple machine translation system (English to French) and explore locality sensitive hashing (LSH) for fast document search.

## 1. Word Embeddings Data for English and French Words
We'll create small synthetic word embedding dictionaries and English-French mapping dictionaries.

In [1]:
import numpy as np
import random
import string
import matplotlib.pyplot as plt

In [2]:
# Create dummy English and French embeddings (10 words, 8D vectors)
words_en = ['cat', 'dog', 'house', 'car', 'tree', 'book', 'apple', 'water', 'sun', 'moon']
words_fr = ['chat', 'chien', 'maison', 'voiture', 'arbre', 'livre', 'pomme', 'eau', 'soleil', 'lune']
en_embeddings_subset = {w: np.random.randn(8) for w in words_en}
fr_embeddings_subset = {w: np.random.randn(8) for w in words_fr}
# English to French mapping
en_fr_train = dict(zip(words_en, words_fr))
en_fr_test = dict(zip(words_en[::-1], words_fr[::-1]))  # reversed for test set

## 1.1 Generate Embedding and Transform Matrices
Let's implement the function to get aligned matrices for translation.

In [3]:
def get_matrices(en_fr, french_vecs, english_vecs):
    X_l, Y_l = [], []
    for en_word, fr_word in en_fr.items():
        if en_word in english_vecs and fr_word in french_vecs:
            X_l.append(english_vecs[en_word])
            Y_l.append(french_vecs[fr_word])
    X = np.vstack(X_l)
    Y = np.vstack(Y_l)
    return X, Y

X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)
X_test, Y_test = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)
print('X_train shape:', X_train.shape, 'Y_train shape:', Y_train.shape)

X_train shape: (10, 8) Y_train shape: (10, 8)


## 2. Translation as Linear Transformation
Let's define the loss, gradient, and alignment functions.

In [4]:
def compute_loss(X, Y, R):
    m = X.shape[0]
    diff = X @ R - Y
    loss = np.sum(diff ** 2) / m
    return loss

def compute_gradient(X, Y, R):
    m = X.shape[0]
    grad = (2 / m) * (X.T @ (X @ R - Y))
    return grad

def align_embeddings(X, Y, train_steps=100, learning_rate=0.01, verbose=True):
    np.random.seed(42)
    R = np.random.randn(X.shape[1], X.shape[1])
    for i in range(train_steps):
        if verbose and i % 25 == 0:
            print(f'loss at iteration {i} is: {compute_loss(X, Y, R):.4f}')
        grad = compute_gradient(X, Y, R)
        R -= learning_rate * grad
    return R

R_train = align_embeddings(X_train, Y_train, train_steps=100, learning_rate=0.01)

loss at iteration 0 is: 66.0740
loss at iteration 25 is: 14.9352
loss at iteration 50 is: 7.5385
loss at iteration 75 is: 5.2592


## 2.2 - Testing the Translation
Let's implement nearest neighbor search and test translation accuracy.

In [5]:
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def nearest_neighbor(v, candidates, k=1):
    sims = [cosine_similarity(v, row) for row in candidates]
    sorted_ids = np.argsort(sims)[::-1]
    return sorted_ids[:k]

def test_vocabulary(X, Y, R):
    pred = X @ R
    num_correct = 0
    for i in range(len(pred)):
        pred_idx = nearest_neighbor(pred[i], Y, k=1)[0]
        if pred_idx == i:
            num_correct += 1
    accuracy = num_correct / X.shape[0]
    return accuracy

acc = test_vocabulary(X_test, Y_test, R_train)
print(f'Accuracy on test set: {acc:.2f}')

Accuracy on test set: 0.70


## 3. LSH and Document Search
Let's create a dummy tweet dataset and implement LSH for fast similarity search.

In [6]:
# Dummy tweet dataset
all_tweets = [
    'I love cats and dogs',
    'The sun is bright',
    'Reading a good book',
    'Water is life',
    'Driving my car to the house',
    'The apple is on the tree',
    'The moon and the stars',
    'My dog drinks water',
    'A cat in the house',
    'Books are great'
]

In [7]:
# Simple tweet processor
def process_tweet(tweet):
    return [w.strip(string.punctuation).lower() for w in tweet.split() if w.strip(string.punctuation)]

In [8]:
def get_document_embedding(tweet, en_embeddings):
    doc_embedding = np.zeros(8)
    for word in process_tweet(tweet):
        if word in en_embeddings:
            doc_embedding += en_embeddings[word]
    return doc_embedding

def get_document_vecs(all_docs, en_embeddings):
    ind2Doc_dict = {}
    document_vec_l = []
    for i, doc in enumerate(all_docs):
        doc_embedding = get_document_embedding(doc, en_embeddings)
        ind2Doc_dict[i] = doc_embedding
        document_vec_l.append(doc_embedding)
    document_vec_matrix = np.vstack(document_vec_l)
    return document_vec_matrix, ind2Doc_dict

document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)
print('Document vectors shape:', document_vecs.shape)

Document vectors shape: (10, 8)


### LSH: Hashing and Fast Nearest Neighbor Search
Let's implement LSH with random planes.

In [9]:
N_VECS = len(all_tweets)
N_DIMS = document_vecs.shape[1]
N_PLANES = 4
N_UNIVERSES = 3
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES)) for _ in range(N_UNIVERSES)]

def hash_value_of_vector(v, planes):
    dot_product = np.dot(v, planes)
    h = (np.sign(dot_product) >= 0).astype(int).flatten()
    hash_value = 0
    for i in range(len(h)):
        hash_value += (2 ** i) * h[i]
    return int(hash_value)

def make_hash_table(vecs, planes):
    num_buckets = 2 ** planes.shape[1]
    hash_table = {i: [] for i in range(num_buckets)}
    id_table = {i: [] for i in range(num_buckets)}
    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v, planes)
        hash_table[h].append(v)
        id_table[h].append(i)
    return hash_table, id_table

hash_tables = []
id_tables = []
for universe_id in range(N_UNIVERSES):
    planes = planes_l[universe_id]
    hash_table, id_table = make_hash_table(document_vecs, planes)
    hash_tables.append(hash_table)
    id_tables.append(id_table)

In [10]:
def approximate_knn(doc_id, v, planes_l, hash_tables, id_tables, k=1, num_universes_to_use=3):
    vecs_to_consider_l = []
    ids_to_consider_l = []
    ids_to_consider_set = set()
    for universe_id in range(num_universes_to_use):
        planes = planes_l[universe_id]
        hash_value = hash_value_of_vector(v, planes)
        hash_table = hash_tables[universe_id]
        id_table = id_tables[universe_id]
        document_vectors_l = hash_table[hash_value]
        new_ids_to_consider = id_table[hash_value]
        for i, new_id in enumerate(new_ids_to_consider):
            if doc_id == new_id:
                continue
            if new_id not in ids_to_consider_set:
                document_vector_at_i = document_vectors_l[i]
                vecs_to_consider_l.append(document_vector_at_i)
                ids_to_consider_l.append(new_id)
                ids_to_consider_set.add(new_id)
    if not vecs_to_consider_l:
        return []
    vecs_to_consider_arr = np.array(vecs_to_consider_l)
    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)
    nearest_neighbor_ids = [ids_to_consider_l[idx] for idx in nearest_neighbor_idx_l]
    return nearest_neighbor_ids

In [11]:
# Example: Find similar tweets
doc_id = 0
vec_to_search = document_vecs[doc_id]
neighbor_ids = approximate_knn(doc_id, vec_to_search, planes_l, hash_tables, id_tables, k=2, num_universes_to_use=3)
print(f'Original tweet: {all_tweets[doc_id]}')
for nid in neighbor_ids:
    print(f'Similar tweet: {all_tweets[nid]}')

Original tweet: I love cats and dogs
Similar tweet: Books are great


C:\Users\Dell\AppData\Local\Temp\ipykernel_15264\2437359634.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
